# May tagging
This notebook is does the tagging for May

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts
from phoenix.common import utils
from phoenix.tag import normalise
from phoenix.tag import feature
from phoenix.tag import data_pull
from phoenix.tag import export

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parametrise the run execution date.
# Format of the run date
RUN_DATE_FORMAT = "%Y-%m-%d"
# This can be overwritten at execution time by Papermill to enable historic runs and backfills etc.
RUN_DATE = datetime.datetime.today().strftime(RUN_DATE_FORMAT)

# Set Artefacts URL
ARTIFACTS_BASE_URL = f"{artifacts.urls.get_local()}{RUN_DATE}/"

# Input CSV
FB_POSTS_INPUT_FOLDER_CSV = f"{artifacts.urls.get_local()}input_csvs/facebook-ct-may/"
TWEETS_INPUT_FOLDER_JSON = f"{artifacts.urls.get_local()}input_csvs/twitter-may/"

In [ ]:
# Display params.
print(
ARTIFACTS_BASE_URL,
FB_POSTS_INPUT_FOLDER_CSV,
TWEETS_INPUT_FOLDER_JSON,
RUN_DATE,
sep='\n',
)

In [ ]:
# %env DASK_CLUSTER_IP=tcp://127.0.0.1:45143

In [ ]:
utils.dask_global_init()

## Facebook Posts

In [ ]:
posts_df = data_pull.crowdtangle_csvs(FB_POSTS_INPUT_FOLDER_CSV)

In [ ]:
posts_df.head()

In [ ]:
artifacts.dataframes.persist(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "normalised_posts"), posts_df)

In [ ]:
posts_df.shape

In [ ]:
posts_df = normalise.execute(posts_df)

In [ ]:
lang_dist = normalise.language_distribution(posts_df)
lang_dist

In [ ]:
posts_features_df = feature.features(posts_df)

In [ ]:
posts_features = feature.explode_features(posts_features_df)

In [ ]:
posts_features.shape

In [ ]:
key_posts, posts_features_has = feature.get_key_items(posts_features)

In [ ]:
key_posts.head()

In [ ]:
posts_to_scrape = export.get_posts_to_scrape(key_posts)
posts_to_scrape.shape

In [ ]:
posts_to_scrape.head()

In [ ]:
with tentaclio.open(ARTIFACTS_BASE_URL + "posts_to_scrape.csv", "w") as fb:
    posts_to_scrape.to_csv(fb)

In [ ]:
lang_dist = normalise.language_distribution(key_posts)
lang_dist

In [ ]:
artifacts.dataframes.persist(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "posts_features"), posts_features[["index", "facebook_id", "features", "features_count", "language"]])

## Tweets

In [ ]:
tweets_df = data_pull.twitter_json(TWEETS_INPUT_FOLDER_JSON)

In [ ]:
tweets_df.head()

In [ ]:
tweets_df = normalise.execute(tweets_df, "full_text")

In [ ]:
tweets_df.shape

In [ ]:
lang_dist = normalise.language_distribution(tweets_df)
lang_dist

In [ ]:
tweets_features_df = feature.features(tweets_df)

In [ ]:
tweets_features = feature.explode_features(tweets_features_df)

In [ ]:
key_tweets, tweets_features_has = feature.get_key_items(tweets_features)

In [ ]:
key_tweets.shape

In [ ]:
lang_dist = normalise.language_distribution(key_tweets)
lang_dist

In [ ]:
artifacts.dataframes.persist(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "tweets_features"), tweets_features[["index", "id_str", "features", "features_count", "language"]])

## Join (all)

In [ ]:
all_features = normalise.join_fb_posts_tweets(posts_features_has, tweets_features_has)

In [ ]:
all_features.head()

In [ ]:
all_features.shape

In [ ]:
features_to_label = feature.get_features_to_label(all_features)

In [ ]:
features_to_label.head()

In [ ]:
features_to_label.shape

In [ ]:
with tentaclio.open(ARTIFACTS_BASE_URL + "all_features_to_label.csv", "w") as fb:
    features_to_label.to_csv(fb)

In [ ]:
tweets_features_to_label = feature.get_features_to_label(all_features[all_features["object_type"] == "tweets"])
with tentaclio.open(ARTIFACTS_BASE_URL + "tweets_features_to_label.csv", "w") as fb:
    tweets_features_to_label.to_csv(fb)

In [ ]:
posts_features_to_label = feature.get_features_to_label(all_features[all_features["object_type"] == "facebook_posts"])
with tentaclio.open(ARTIFACTS_BASE_URL + "posts_features_to_label.csv", "w") as fb:
    posts_features_to_label.to_csv(fb)

In [ ]:
key_tweets_features_to_label = feature.get_features_to_label(all_features[(all_features["object_type"] == "tweets") & (all_features["has_key_feature"] == True)])
with tentaclio.open(ARTIFACTS_BASE_URL + "key_tweets_features_to_label.csv", "w") as fb:
    key_tweets_features_to_label.to_csv(fb)

In [ ]:
key_posts_features_to_label = feature.get_features_to_label(all_features[(all_features["object_type"] == "facebook_posts") & (all_features["has_key_feature"] == True)])
with tentaclio.open(ARTIFACTS_BASE_URL + "key_posts_features_to_label.csv", "w") as fb:
    key_posts_features_to_label.to_csv(fb)

In [ ]:
with tentaclio.open(ARTIFACTS_BASE_URL + "posts_with_key_features.csv", "w") as fb:
    key_posts.to_csv(fb)

In [ ]:
with tentaclio.open(ARTIFACTS_BASE_URL + "tweets_with_key_features.csv", "w") as fb:
    key_tweets.to_csv(fb)